In [93]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.ticker import FuncFormatter
from matplotlib.widgets import Slider
from scipy.integrate import solve_ivp

plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('font', size=10)
plt.rc('axes', titlesize=16)
plt.rc('axes', labelsize=15)
plt.rc('legend', fontsize=10)
plt.rcParams["figure.figsize"] = (10, 7)

In [ ]:
%matplotlib qt5

In [176]:
class Solver:
    def __init__(self,system,K,initial,comps,T) -> None:
        self.system = system
        self.K=K
        self.initial = initial
        self.T =T

        comps = comps.replace('[', '').replace(']', '')
        self.comp={}
        for i, key in enumerate(comps.split(', ')):
            self.comp[key]=i

    def solve(self,initial=None,K=None):
        init =  initial if initial is not None else self.initial
        k = K if K is not None else self.K

        solution = solve_ivp(
            fun=self.system,
            t_span=[0, T.max()],
            y0=init,
            args=(k,),
            dense_output=True,
        )
        self.y = solution.sol(self.T)
    def __getitem__(self,value):
        return self.y[self.comp[value]]

class Differ:
    def __init__(self, y1, y2) -> None:
        self.y1: Solver = y1
        self.y2: Solver = y2

    def __getitem__(self, comp):
        sigma = 1e-9
        return (self.y1[comp] - self.y2[comp]) / (self.y1[comp] + sigma) * 100

    def solve(self,initial1=None,K1=None,initial2=None,K2=None):
        self.y1.solve(initial=initial1,K=K1)
        self.y2.solve(initial=initial2,K=K2)

In [177]:
T = np.linspace(0, 0.0001, 100)
K = dict(
    light=0.0001,
    Ke=10**9,
    KH=10**9,
    Kr=10**9,
    Kp=0.001,
    KqH=2000,
    Kdisp=10**9,
    Kph=10**-5,
    Ks=2,
    Kd=0.05,
    Kc=1,
    KrD=10**9,
)

In [188]:
def show_plt(y: Solver):
    fig, ax = plt.subplots()
    gs = plt.GridSpec(2, 2, figure=fig)
    fig.subplots_adjust(left=0.25, right=0.99, bottom=0.1, top=0.95, hspace=0.1, wspace=0.1)
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

    scal = FuncFormatter(lambda x, pos: f'{x*1000: .2f}')
    K = y.K
    y.solve()

    plots = {}

    base_ax = plt.subplot(gs[0, 0:2])
    for c in ['Q', 'DH', 'QHH']:
        plots[c] = base_ax.plot(T, y[c], label=c)[0]
    base_ax.legend()
    base_ax.xaxis.set_major_formatter(scal)

    d_ax = plt.subplot(gs[1, 0])
    plots['D'] = d_ax.plot(T, y['D'], label='D')[0]
    d_ax.legend()
    d_ax.xaxis.set_major_formatter(scal)

    other_ax = plt.subplot(gs[1, 1])
    other_ax.xaxis.set_major_formatter(scal)
    for c in ['QH', 'QHD']:
        plots[c] = other_ax.plot(T, y[c], label=c)[0]
    other_ax.legend()
    other_ax.xaxis.set_major_formatter(scal)

    sliders = {}
    for i, k in enumerate(K):
        k_axes = fig.add_axes([0.05, 0.95 - 0.05 * i, 0.15, 0.05])
        amp_slider = Slider(
            ax=k_axes,
            label=k,
            valmin=-3,
            valmax=3,
            valinit=0,
            valstep=0.1,
            orientation="horizontal",
        )
        sliders[k] = amp_slider

    def update(val):
        k = K.copy()
        for h in K:
            k[h] = K[h] * 10 ** sliders[h].val

        y.solve(K=k)
        for c, plot in plots.items():
            plot.set_ydata(y[c])
        fig.canvas.draw_idle()

    for h in K:
        sliders[h].on_changed(update)

    return fig


def get_dif(y1: Solver, y2: Solver, comp):
    sigma = 1e-9
    return (y1[comp] - y2[comp]) / (y1[comp] + sigma) * 100


def show_diff(y1: Solver, y2: Solver):
    fig, ax = plt.subplots()
    gs = plt.GridSpec(2, 2, figure=fig)
    fig.subplots_adjust(left=0.25, right=0.99, bottom=0.1, top=0.95, hspace=0.1, wspace=0.1)
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

    scal = FuncFormatter(lambda x, pos: f'{x*1000: .2f}')
    K = y1.K

    y = Differ(y1, y2)
    y.solve()

    plots = {}

    base_ax = plt.subplot(gs[0, 0:2])
    for c in ['Q', 'DH', 'QHH']:
        plots[c] = base_ax.plot(T, y[c], label=c)[0]
    base_ax.legend()
    base_ax.xaxis.set_major_formatter(scal)
    base_ax.set_ylim([-50, 50])

    d_ax = plt.subplot(gs[1, 0])
    plots['D'] = d_ax.plot(T, y['D'], label='D')[0]
    d_ax.legend()
    d_ax.xaxis.set_major_formatter(scal)
    d_ax.set_ylim([-50, 50])

    other_ax = plt.subplot(gs[1, 1])
    other_ax.xaxis.set_major_formatter(scal)
    for c in ['QH', 'QHD']:
        plots[c] = other_ax.plot(T, y[c], label=c)[0]
    other_ax.legend()
    other_ax.xaxis.set_major_formatter(scal)
    other_ax.set_ylim([-50, 50])

    sliders = {}
    for i, k in enumerate(K):
        k_axes = fig.add_axes([0.05, 0.95 - 0.05 * i, 0.15, 0.05])
        amp_slider = Slider(
            ax=k_axes,
            label=k,
            valmin=-3,
            valmax=3,
            valinit=0,
            valstep=0.1,
            orientation="horizontal",
        )
        sliders[k] = amp_slider

    def update(val):
        k = K.copy()
        for h in K:
            k[h] = K[h] * 10 ** sliders[h].val

        y.solve(K1=k, K2=k)
        for c, plot in plots.items():
            plot.set_ydata(y[c])
        fig.canvas.draw_idle()

    for h in K:
        sliders[h].on_changed(update)

    return fig


In [189]:
# consts0
def system0(t, C, k=K):
    [Q, DH, QHH, D, QHD, QH, QD, Qm, DHp] = C
    re = k['Ke'] * k['light'] * Q * DH
    rH = k['KH'] * Qm * DHp
    rr = k['Kr'] * QH * D
    rp = k['Kp'] * QHD
    rqH = k['KqH'] * Q * QHH
    rdisp = k['Kdisp'] * QH * QH
    rph = k['Kph'] * Q
    rs = k['Ks'] * Q * QHD
    rd = k['Kd'] * QD
    rc = k['Kc'] * Q * D
    rrD = k['KrD'] * D * D

    Q = -re - rqH - rph - rs - rc + rdisp + rd
    DH = -re
    QHH = -rqH + rp + rdisp
    D = -rr - rc - 2 * rrD + rH + rd
    QHD = -rp - rs + rr
    QH = -rr - 2 * rdisp + rH + 2 * rqH + rs
    QD = -rd + rs + rc
    Qm = -rH + re
    DHp = -rH + re

    return [Q, DH, QHH, D, QHD, QH, QD, Qm, DHp]


c0 = '[Q, DH, QHH, D, QHD, QH, QD, Qm, DHp]'
initial0 = [1, 1, 0, 0, 0, 0, 0, 0, 0]

In [185]:
show_plt(Solver(system0,K,initial0,c0,T))
1

1

In [190]:
def system1(t, C, k=K):
    [Q, DH, QHH, D, QHD, QH, QD] = C
    re = k['Ke'] * k['light'] * Q * DH
    rr = k['Kr'] * QH * D
    rp = k['Kp'] * QHD
    rqH = k['KqH'] * Q * QHH
    rdisp = k['Kdisp'] * QH * QH
    rph = k['Kph'] * Q
    rs = k['Ks'] * Q * QHD
    rd = k['Kd'] * QD
    rc = k['Kc'] * Q * D
    rrD = k['KrD'] * D * D

    Q = -re - rqH - rph - rs - rc + rdisp + rd
    DH = -re
    QHH = -rqH + rp + rdisp
    D = -rr - rc - 2 *rrD + re + rd
    QHD = -rp - rs + rr
    QH = -rr - 2 *rdisp + re + 2 *rqH + rs
    QD = -rd + rs + rc

    return [Q, DH, QHH, D, QHD, QH, QD]

c1 = '[Q, DH, QHH, D, QHD, QH, QD]'
initial1 = [1, 1, 0, 0, 0, 0, 0]

In [193]:
show_diff(Solver(system0,K,initial0,c0,T),Solver(system1,K,initial1,c1,T) )
1

1